### Basic Trajectory Prediction Workflow

This workflow models trajectory using the TorchDYN library for Hybrid NeuralODE using LSTM base

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from torchdyn.models import NeuralODE
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
# ------------------ DATA ------------------ #
class FullSequenceDataLoader:
    def __init__(self, dataset_path='../../SEIR_CSV.csv', batch_size=1, test_size=0.2, scaler_path='scaler.pkl'):
        self.dataset_path = dataset_path
        self.batch_size = batch_size
        self.test_size = test_size
        self.scaler_path = scaler_path

        self.scaler = self.load_or_create_scaler()
        self.data = self.load_data()
        self.train_loader, self.test_loader = self.create_dataloaders()

    def load_or_create_scaler(self):
        try:
            with open(self.scaler_path, 'rb') as f:
                scaler = pickle.load(f)
            print("✅ Scaler loaded successfully!")
        except (FileNotFoundError, EOFError):
            scaler = MinMaxScaler()
            print("⚠️ Scaler not found. Creating a new one.")
        return scaler

    def save_scaler(self):
        with open(self.scaler_path, 'wb') as f:
            pickle.dump(self.scaler, f)
        print("✅ Scaler saved successfully!")

    def load_data(self):
        df = pd.read_csv(self.dataset_path).astype(float)
        df_scaled = self.scaler.fit_transform(df)
        self.save_scaler()
        return df_scaled

    def create_dataloaders(self):
        sequence = torch.tensor(self.data, dtype=torch.float32).unsqueeze(0)  # Shape: (1, T, F)
        length = sequence.shape[1]
        split = int(length * (1 - self.test_size))

        train_seq = sequence[:, :split, :]
        test_seq = sequence[:, split:, :]

        train_loader = DataLoader(TensorDataset(train_seq), batch_size=self.batch_size, shuffle=False)
        test_loader = DataLoader(TensorDataset(test_seq), batch_size=self.batch_size, shuffle=False)
        return train_loader, test_loader

In [3]:
# ------------------ MODEL ------------------ #
class LSTMHybrid(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, t, y, args=None):
        if y.dim() == 2:
            y = y.unsqueeze(1)
        out, _ = self.lstm(y)
        return self.fc(out[:, -1, :])

class HybridODE(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super().__init__()
        self.ode_func = LSTMHybrid(input_dim, hidden_dim, num_layers, output_dim)
        self.ode_solver = NeuralODE(self.ode_func, solver="tsit5", sensitivity="adjoint")

    def forward(self, y0, t, args=None):
        # ✅ DO NOT modify batch dim; let it be [batch, features]
        t_eval, sol = self.ode_solver(y0, t)
        return sol  # shape: [time, batch, features]



In [4]:
# ------------------ TRAINING ------------------ #
def train_neural_ode(dataset_path='../../SEIR_CSV.csv', batch_size=25, epochs=100, lr=0.01):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    data_loader = FullSequenceDataLoader(dataset_path, batch_size)
    train_loader, test_loader = data_loader.train_loader, data_loader.test_loader

    input_dim, hidden_dim, num_layers, output_dim = 7, 64, 2, 7
    model = HybridODE(input_dim, hidden_dim, num_layers, output_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    best_loss = float('inf')
    best_model_state = None

    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for (sequence,) in train_loader:
            sequence = sequence.to(device)
            y0 = sequence[:, 0, :]
            t = torch.linspace(0, 1, sequence.shape[1], device=device)
            targets = sequence.squeeze(0)  # Shape: (T, F)
            # print("sequences shape:", sequence.shape)
            # print("targets shape:", targets.shape)
            
            optimizer.zero_grad()
            outputs = model(y0, t)        # [800, 1, 7]
            outputs = outputs.squeeze(1) # [800, 7] ← batch dim gone
            loss = loss_fn(outputs, targets)
            
            # print("y0 shape:", y0.shape)
            # print("outputs shape:", outputs.shape)
            # print("targets shape:", targets.shape)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch + 1}, Loss: {avg_loss:.6f}")

        if avg_loss < best_loss:
            best_loss = avg_loss
            best_model_state = model.state_dict()

    model.load_state_dict(best_model_state)
    return model, test_loader

In [5]:
# ------------------ VISUALIZATION ------------------ #
def visualize_results(model, test_loader, scaler_path='scaler.pkl', output_dir="visualizations"):
    os.makedirs(output_dir, exist_ok=True)
    with open(scaler_path, 'rb') as f:
        scaler = pickle.load(f)

    model.eval()
    all_actual, all_predicted = [], []
    with torch.no_grad():
        for (sequence,) in test_loader:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            sequence = sequence.to(device)
            y0 = sequence[:, 0, :]
            t = torch.linspace(0, 1, sequence.shape[1], device=device)
            prediction = model(y0, t).cpu().numpy()
            actual = sequence.cpu().squeeze(0).numpy()
            all_actual.append(actual)
            all_predicted.append(prediction)

    # Flatten predictions and targets for inverse transform
    actual_flat = np.concatenate(all_actual, axis=0).reshape(-1, all_actual[0].shape[-1])     # (samples, features)
    predicted_flat = np.concatenate(all_predicted, axis=0).reshape(-1, all_predicted[0].shape[-1])  # (samples, features)

    # Inverse transform
    actual_flat = scaler.inverse_transform(actual_flat)
    predicted_flat = scaler.inverse_transform(predicted_flat)


    mse = mean_squared_error(actual_flat, predicted_flat, multioutput='raw_values')
    mae = mean_absolute_error(actual_flat, predicted_flat, multioutput='raw_values')
    rmse = np.sqrt(mse)
    r2 = r2_score(actual_flat, predicted_flat, multioutput='raw_values')

    metrics = pd.DataFrame({
        "Feature": scaler.feature_names_in_,
        "MSE": mse,
        "RMSE": rmse,
        "MAE": mae,
        "R2 Score": r2
    })
    metrics.to_csv(os.path.join(output_dir, "metrics.csv"), index=False)

    for i, feature in enumerate(scaler.feature_names_in_):
        plt.figure(figsize=(10, 4))
        plt.plot(actual_flat[:, i], label="Actual", linestyle="dashed")
        plt.plot(predicted_flat[:, i], label="Predicted")
        plt.title(f"Actual vs Predicted ({feature}) - Full Trajectory")
        plt.xlabel("Time Steps")
        plt.ylabel("Value")
        plt.grid(True)
        plt.legend()
        plt.savefig(os.path.join(output_dir, f"{feature}_comparison.png"))
        plt.close()
        
    print(metrics)

In [6]:
model, test_loader = train_neural_ode()
visualize_results(model, test_loader)

✅ Scaler loaded successfully!
✅ Scaler saved successfully!
Epoch 1, Loss: 0.192242
Epoch 2, Loss: 0.174840
Epoch 3, Loss: 0.156920
Epoch 4, Loss: 0.136405
Epoch 5, Loss: 0.112000
Epoch 6, Loss: 0.083826
Epoch 7, Loss: 0.055097
Epoch 8, Loss: 0.036393
Epoch 9, Loss: 0.048273
Epoch 10, Loss: 0.056424
Epoch 11, Loss: 0.045515
Epoch 12, Loss: 0.033570
Epoch 13, Loss: 0.028754
Epoch 14, Loss: 0.029812
Epoch 15, Loss: 0.033198
Epoch 16, Loss: 0.036322
Epoch 17, Loss: 0.038015
Epoch 18, Loss: 0.038100
Epoch 19, Loss: 0.036917
Epoch 20, Loss: 0.035005
Epoch 21, Loss: 0.032933
Epoch 22, Loss: 0.031189
Epoch 23, Loss: 0.030096
Epoch 24, Loss: 0.029745
Epoch 25, Loss: 0.029967
Epoch 26, Loss: 0.030397
Epoch 27, Loss: 0.030660
Epoch 28, Loss: 0.030567
Epoch 29, Loss: 0.030182
Epoch 30, Loss: 0.029719
Epoch 31, Loss: 0.029383
Epoch 32, Loss: 0.029266
Epoch 33, Loss: 0.029343
Epoch 34, Loss: 0.029514
Epoch 35, Loss: 0.029665
Epoch 36, Loss: 0.029708
Epoch 37, Loss: 0.029602
Epoch 38, Loss: 0.029357
